
**Modified by Roke Manor Research to support a runnable notebook - with additions to show use of protocol buffer compilation.**

In [ ]:
from sys import version

version

### Installing proto buffers
```sudo apt-get install protobuf-compiler```

```protoc -I=aki_predictions/ehr_prediction_modeling --python_out=aki_predictions/ehr_prediction_modeling aki_predictions/ehr_prediction_modeling/proto/*.proto```

### Import dependencies

In this notebook, we load and display data at various processing stages: raw `FakeRecord`, sequential `FakePatient`, vectorized `GenericEventSequence`, and sparse encoded `tf.SequenceExample`

In [ ]:
import tensorflow.compat.v1 as tf

# @title Imports
# Protocol buffers must first be compiled using `protoc`: see https://developers.google.com/protocol-buffers/docs/pythontutorial for details.from ehr_prediction_modeling.proto import fake_records_pb2
from aki_predictions.ehr_prediction_modeling.proto import (  # Missing from original notebook.
    fake_generic_representation_pb2,
    fake_patient_pb2,
    fake_records_pb2,
)

tf.disable_v2_behavior()
import os

In [ ]:
print(os.getcwd())
data_dirpath = os.getcwd()  # "path/to/directory/that/stores/data"  # @param

### Read Records

In [ ]:
raw_data_path = os.path.join(data_dirpath, "fake_raw_records.pb")

with open(raw_data_path, "rb") as f:
    records = fake_records_pb2.FakeRecords.FromString(f.read()).records

In [ ]:
print(
    f"There are {len(records)} records in the fake dataset.\nOne example is:\n\n{records[0]}"
)

### Read Patients

In [ ]:
patient_path = os.path.join(data_dirpath, "fake_patients.pb")

with open(patient_path, "rb") as f:
    patients = fake_patient_pb2.FakePatients.FromString(f.read()).patients

In [ ]:
print(
    f"There are {len(patients)} patients in the fake dataset.\nOne example is:\n\n{patients[0]}"
)

In [ ]:
all_admissions = []
for patient in patients:
    for episode in patient.episodes:
        if episode.WhichOneof("episode_type") == "admission":
            all_admissions.append(episode.admission)

print(
    f"There are {len(all_admissions)} admissions in the fake dataset.\nOne example is:\n\n{all_admissions[0]}"
)

In [ ]:
all_clinical_events = []
for patient in patients:
    for episode in patient.episodes:
        if episode.WhichOneof("episode_type") == "admission":
            for event in episode.admission.clinical_events:
                all_clinical_events.append(event)

print(
    f"There are {len(all_clinical_events)} clinical events in admission in the fake dataset.\nOne example is:\n\n{all_clinical_events[0]}"
)

### Read GenericEventSequence

In [ ]:
vectorized_path = os.path.join(data_dirpath, "vectorized", "fake_vectorized_samples.pb")

with open(vectorized_path, "rb") as f:
    event_sequences = (
        fake_generic_representation_pb2.FakeGenericEventSequences.FromString(
            f.read()
        ).generic_event_sequences
    )

In [ ]:
print(
    f"There are {len(event_sequences)} event sequences in the fake dataset.\nOne example is:\n\n{event_sequences[0]}"
)

### `tf.SequenceExample`

In [ ]:
split = "train"  # @param
seqex_path = os.path.join(data_dirpath, f"standardize/{split}.tfrecords")
filenames = [seqex_path]
raw_dataset = tf.data.TFRecordDataset(filenames)
iterator = raw_dataset.make_initializable_iterator()
init = tf.initialize_all_variables()
batch = iterator.get_next()

In [ ]:
all_seqexs = []
with tf.train.MonitoredTrainingSession() as sess:
    sess.run(iterator.initializer)
    sess.run(init)
    while True:
        all_seqexs.append(tf.train.SequenceExample.FromString(sess.run(batch)))

In [ ]:
print(
    f"There are {len(all_seqexs)} sequence examples in the fake dataset for {split} split.\nOne example is:\n\n{all_seqexs[0]}"
)